# Solicitações Dados Radar SP

Objetivo: Obter arquivo JSON de cada viagem_id, com discriminação das rotas e seus arcos 

http://dadosradares.prefeitura.sp.gov.br/

In [ ]:
import requests
import json
import os
from tqdm import tqdm

# 1 - Códigos dos radares e Datas com dados registrados

## getLocais

Obtem (e arquiva) os códigos dos radares (alguns radares possuem mais de 1 código)

In [ ]:
# Obtenção dos códigos de cada radar
r = requests.get("http://<emailregistrado>:<senha>@dadosradares.prefeitura.sp.gov.br/getLocais/")
try: codigos = open("codigos_radares.txt").read().splitlines()
except:
    codigos = []
    for i in r.json():
        for j in i["codigos"]: codigos.append(j)
            
    # Salvar lista como codigos_radares
    with open("codigos_radares.txt", "w") as outfile:
        outfile.write("\n".join(str(item) for item in codigos))

len(codigos)

## getDatas

Obtem (e arquiva) lista com todas as datas disponíveis

In [ ]:
# Obter lista de todas as datas

try: datas = open("datas.txt").read().splitlines()
except:
    r_2 = requests.get("http://dadosradares.prefeitura.sp.gov.br/getDatas/")
    datas = r_2.json()["results"]
    
    # Salvar lista com as datas
    with open("datas.txt", "w") as outfile:
        outfile.write("\n".join(str(item) for item in datas))

#datas # Para visualizar todas as datas disponíveis

# 2 - Obtenção de lista com viagem_id

## getViagens

Acessa o getViagens usando um codigo ou lista de códigos do radar, data específica e horas ou range de horas para obter uma lista com viagem_id

In [ ]:
# Definição do período a ser analisado

# http://dadosradares.prefeitura.sp.gov.br/getViagens/<YYYY-MM-DD>/<hora>/<codigo>/  # Modelo para solicitação
url = "http://<emailregistrado>:<senha>@dadosradares.prefeitura.sp.gov.br/getViagens/"
data = "16/02/2019"  # Data avaliada
codigo = ["2654","5441","5317","2448","2643","5208","4388","4395","4401","6825","1201","4594","5418"] # código único ["<codigo>"] ou lista codigos[<index>:<index>] ou ["codigo_1","codigo_2","codigo_3"]  
hora = range(7,11)  # Hora específica (15) ou período (range(15,18))

In [ ]:
# Gera uma lista com todos os id's de viagem (viagem_id) e um modelo JSON (r_3.json())

viagem_id = []  # Lista final com ids conseguidos
YYYY = data[6:10]
MM = data[3:5]
DD = data[0:2]
for c in codigo:
    if type(hora) != range:
        h = hora
        url_2 = url+YYYY+"-"+MM+"-"+DD+"/"+str(h)+"/"+c  # URL final para cada um dos dados
        print (url_2) # Caso queira checar manualmente os links para entender a requisição do json
        r_3 = requests.get(url_2)       
        
        if ((r_3.json()["total"] < 1) or (r_3.json()["total"] == None)): continue # Desconsidera quando não há registros
        
        for k in  r_3.json()["result"]:
            if k["id"] not in viagem_id: viagem_id.append(k["id"])
            viagem_id.append(k["id"])
        print("codigo do radar:",c,"\n"
              "hora da viagem:",h,"\n"
              "total de viagens:",r_3.json()["total"])
            
    else:
        for h in hora:
            url_2 = url+YYYY+"-"+MM+"-"+DD+"/"+str(h)+"/"+c  # URL final para cada um dos dados
            print (url_2) # Caso queira checar manualmente os links para entender a requisição do json
            r_3 = requests.get(url_2)       
            
            if ((r_3.json()["total"] < 1) or (r_3.json()["total"] == None)): continue # Desconsidera qunado não há registros
                
            for k in  r_3.json()["result"]:
                if k["id"] not in viagem_id: viagem_id.append(k["id"])
            print("codigo do radar:",c,"\n"
                  "hora da viagem:",h,"\n"
                  "total de viagens:",r_3.json()["total"])
    
    # Caso queira limitar o tamanho da amostra
    if len(viagem_id) > 2000: break


# 3 - Download dos Resultados

## getTrajetos

Obtem viagens e pontos de passagem, extraindo todos os códigos dos radares e horários. 


In [ ]:
# Não rodar quando for obter rotas em bloco, indica quantos pontos de passagem cada viagem_id tem (interromper Kernel -> Interrupt)
count = 0
for i in viagem_id:
    r_4 = requests.get("http://<emailregistrado>:<senha>@dadosradares.prefeitura.sp.gov.br/getTrajetos/"+str(i))
    print("viagem_id:",i,"\n"
        "viagem com",r_4.json()["total"],"pontos de passagem","\n")
    count += 1
    #if count == 30: break


In [ ]:
# Função a ser aplicada em cada elemento da viagem_id
def rota(id):
    dados_radares_dict = {}
    dados_radares = []
    dados_radares.append({"viagem_id":id})
    
    
    # http://dadosradares.prefeitura.sp.gov.br/getTrajetos/<viagem_id>/  # Modelo para solicitação
    r_4 = requests.get("http://<emailregistrado>:<senha>@dadosradares.prefeitura.sp.gov.br/getTrajetos/"+str(id))
    
    # Dicionário com informações específicas salvas em uma lista final a ser exportada e importada pelo script TomTom
    for v in r_4.json()["result"]:
        
        if (str(v["inicio"]) or str(v["final"]) in codigos) == 0:
            print("Rota_"+str(id)+" não é elegível","\n"
                  "Radar",v["inicio"],"ou radar",v["final"],"não está no banco de dados, ver getLocais","\n")
            status = "off"
            break
        
        status = "on"
        dados_radares_dict["radar_inicio"] = v["inicio"]
        dados_radares_dict["radar_final"] = v["final"]
        dados_radares_dict["data_inicio"] =  v["data_inicio"]
        dados_radares_dict["data_final"] = v["data_final"]
        
        # Estrutura loop, que compara os códigos dos radares do getLocais e quando equivalente ao código registrado no arco
        # (inicio ou final) armazena suas coordenadas no dicionário
        for i in r.json():
            for j in i["codigo"]:
                if str(dados_radares_dict["radar_inicio"]) == str(j):
                    dados_radares_dict["lat_inicio"] = i["y"]
                    dados_radares_dict["long_inicio"] = i["x"]
                elif str(dados_radares_dict["radar_final"]) == str(j):
                    dados_radares_dict["lat_final"] = i["y"]
                    dados_radares_dict["long_final"] = i["x"]
        dados_radares.append(dados_radares_dict.copy())
    return dados_radares,status

In [ ]:
viagem_id[756]

In [ ]:
# Cria pasta para armazenamento dos resultados
try:
    foldername_arquivos = r"rotas"
    os.mkdir(foldername_arquivos)
except: 
    print("Pasta já Existente")

viagem_id_erro = []
for i in tqdm(viagem_id[756]):
    dados,status = rota(i)
    
    if status == "off":
        viagem_id_erro.append("rota_"+str(i))
        continue
    
    with open(r"rotas\rota_"+str(i)+".json", "w") as outfile:
        json.dump(dados, outfile)
    

In [ ]:
# Salvando Informações Gerais
print(
"Viagens totais analisadas:",len(viagem_id),"\n"
"Viagens rejeitadas:",len(viagem_id_erro),"\n"
"Viagens salvas:",len(viagem_id)-len(viagem_id_erro))

with open("viagens_rejeitadas.txt", "w") as outfile:
    outfile.write("\n".join(str(item) for item in viagem_id_erro))
